In [1]:
from gurobipy import GRB
from models.SSPMF import SSPMFModel
from models.JGSMF_with_phases import solve_with_phases
from models.JGSMF import JGSMFModel
import pandas as pd
import os
from datetime import datetime

def retrieve_format_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        first_row = lines[0].strip().split()
        num_jobs = int(first_row[0])
        num_tools = int(first_row[1])
        magazine_capacity = int(first_row[2])
        
        job_tools = {job: [] for job in range(1, num_jobs + 1)}
        
        for tool_index, line in enumerate(lines[1:], start=1):
            tool_requirements = [int(x) for x in line.strip().split()]
            for job_index, requirement in enumerate(tool_requirements, start=1):
                if requirement == 1:
                    job_tools[job_index].append(tool_index)
                    
    return num_jobs, num_tools, magazine_capacity, job_tools

directory = '../data/MTSP/Laporte/Tabela4/'
data = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        num_jobs, num_tools, magazine_capacity, job_tools = retrieve_format_data(file_path)
        data.append([filename, num_jobs, num_tools, magazine_capacity, job_tools])

df = pd.DataFrame(data, columns=['filename', 'num_jobs', 'num_tools', 'magazine_capacity', 'job_tools'])
#df = df.sample(frac=0.02).reset_index(drop=True)

In [2]:
jobs = [1,2,3,4,5,6]
tools = [1,2,3]
magazine_capacity = 2
job_tools = {1: [1,2], 2: [1], 3: [2], 4: [1,2], 5: [3], 6: [2,3]}

model = JGSMFModel(jobs, tools, magazine_capacity, job_tools)
model.optimize()
solution = model.get_solution()
print("Job order: ", solution) 

Q:  2
Set parameter Username
Set parameter LicenseID to value 2588882
Academic license - for non-commercial use only - expires 2025-11-22
Model is infeasible
Detached tools: 
None
Active bins: 
No optimal solution found.
None
Jobs position: 
No optimal solution found.
None
Jobs in bins: 
No optimal solution found.
None
Tools flow: 
No optimal solution found.
None
No optimal solution found.
Job order:  None


In [3]:
file_path = '../data/MTSP/Laporte/Tabela4/L11-1.txt'
num_jobs, num_tools, magazine_capacity, job_tools = retrieve_format_data(file_path)
jobs = [x for x in range(1, num_jobs + 1)]
tools = [x for x in range(1, num_tools + 1)]
print("Jobs: ", jobs)
print("Tools: ", tools)
print("Magazine Capacity: ", magazine_capacity)
print("Job Tools: ", job_tools)

model = JGSMFModel(jobs, tools, magazine_capacity, job_tools)
model.optimize()
solution = model.get_solution()
print("Job Order: ", solution)


Jobs:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
Tools:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Magazine Capacity:  15
Job Tools:  {1: [1, 2, 3, 6, 11, 13, 14, 16, 17, 18, 19, 20], 2: [1, 2, 4, 6, 7, 8, 11, 12, 16, 18, 19], 3: [4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20], 4: [1, 3, 4, 5, 9, 13, 14, 17, 18, 19], 5: [4, 5, 7, 8, 12, 13, 14, 16, 18, 20], 6: [1, 2, 3, 4, 5, 6, 8, 10, 11, 14, 15, 17, 20], 7: [1, 3, 5, 6, 7, 8, 9, 11, 12, 13, 15, 18, 19, 20], 8: [1, 2, 3, 4, 5, 7, 8, 9, 12, 14, 15, 16, 17, 20], 9: [1, 2, 3, 5, 8, 9, 11, 12, 13, 14, 15, 16, 17, 20]}
Q:  7
Model is infeasible
Detached tools: 
None
Active bins: 
No optimal solution found.
None
Jobs position: 
No optimal solution found.
None
Jobs in bins: 
No optimal solution found.
None
Tools flow: 
No optimal solution found.
None
No optimal solution found.
Job Order:  None


In [4]:
file_path = '../data/MTSP/Laporte/Tabela4/L10-10.txt'
num_jobs, num_tools, magazine_capacity, job_tools = retrieve_format_data(file_path)
jobs = [x for x in range(1, num_jobs + 1)]
tools = [x for x in range(1, num_tools + 1)]
# print("Jobs: ", jobs)
# print("Tools: ", tools)
# print("Magazine Capacity: ", magazine_capacity)
# print("Job Tools: ", job_tools)

model = JGSMFModel(jobs, tools, magazine_capacity, job_tools)
model.optimize()
solution = model.get_solution()
print("Job Order: ", solution)


Q:  2
Optimal switches: 5
Detached tools: 
Tool 7 is sent to detached from bin 1
Tool 13 is sent to detached from bin 1
Tool 12 is sent to detached from bin 2
Tool 2 is sent to detached from bin 3
Tool 16 is sent to detached from bin 3
Tool 1 is sent to detached from bin 9
Tool 3 is sent to detached from bin 9
Tool 4 is sent to detached from bin 9
Tool 5 is sent to detached from bin 9
Tool 6 is sent to detached from bin 9
Tool 8 is sent to detached from bin 9
Tool 9 is sent to detached from bin 9
Tool 10 is sent to detached from bin 9
Tool 11 is sent to detached from bin 9
Tool 14 is sent to detached from bin 9
Tool 15 is sent to detached from bin 9
Tool 17 is sent to detached from bin 9
Tool 18 is sent to detached from bin 9
Tool 19 is sent to detached from bin 9
Tool 20 is sent to detached from bin 9
None
Active bins: 
[1, 2, 3, 4]
Jobs position: 
Job 1 is in bin 2
Job 2 is in bin 1
Job 3 is in bin 1
Job 4 is in bin 4
Job 5 is in bin 3
Job 6 is in bin 3
Job 7 is in bin 4
Job 8 is in 

In [2]:
directory = '../data/MTSP/Laporte/Tabela4/'
data = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        num_jobs, num_tools, magazine_capacity, job_tools = retrieve_format_data(file_path)
        data.append([num_jobs, num_tools, magazine_capacity, job_tools])

df = pd.DataFrame(data, columns=['num_jobs', 'num_tools', 'magazine_capacity', 'job_tools'])
df = df.sample(frac=0.02).reset_index(drop=True)

In [ ]:
results = []

for index, row in df.iterrows():
    print("Current instance: ", row['filename'])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    #print("Jobs: ", jobs)
    #print("Tools: ", tools)
    #print("Magazine Capacity: ", magazine_capacity)
    #print("Job Tools Requirements: ", job_tools_requirements)

    model = JGSMFModel(jobs, tools, magazine_capacity, job_tools_requirements)

    model.optimize()

    if model.model.status == GRB.OPTIMAL:
        solution_found = True
        solution = model.get_solution()
        print("Job Order: ", solution)
    else:
        solution_found = False
        solution = None

    runtime = model.model.Runtime


    results.append([num_jobs, num_tools, magazine_capacity, solution_found, solution, runtime])

results_df = pd.DataFrame(results, columns=['N', 'M', 'C', 'solution_found', 'solution', 'runtime'])
results_df
        



Current instance:  L1-1.txt
Cliques: [[2, 4, 6, 1], [2, 4, 6, 3], [2, 4, 6, 5], [2, 4, 6, 7], [2, 4, 6, 8], [2, 4, 6, 9]]
Optimal switches: 12
Job Order:  [6, 3, 8, 1, 2, 7, 5, 9, 4]
Current instance:  L1-10.txt
Cliques: [[6, 7, 9, 4], [6, 7, 9, 5, 8], [6, 7, 9, 5, 3, 1], [6, 7, 9, 5, 3, 2]]
Optimal switches: 12
Job Order:  [2, 4, 8, 3, 1, 5, 6, 7, 9]
Current instance:  L1-2.txt
Cliques: [[1, 3, 5, 6, 7, 8], [1, 3, 5, 6, 7, 2, 9], [1, 3, 5, 6, 7, 2, 4]]
Optimal switches: 13
Job Order:  [7, 4, 1, 5, 6, 9, 3, 2, 8]
Current instance:  L1-3.txt
Cliques: [[1, 3, 4, 9, 2, 8, 7], [1, 3, 4, 9, 2, 6], [1, 3, 4, 9, 5, 8], [1, 3, 4, 9, 5, 6]]
Optimal switches: 13
Job Order:  [4, 6, 8, 7, 9, 1, 2, 5, 3]
Current instance:  L1-4.txt
Cliques: [[2, 5, 6, 7, 1, 8, 9, 3], [2, 5, 6, 7, 4]]
Optimal switches: 13
Job Order:  [9, 6, 8, 7, 5, 3, 2, 1, 4]
Current instance:  L1-5.txt
Cliques: [[3, 2, 8], [3, 2, 1, 6], [3, 2, 9], [3, 2, 7], [3, 4], [3, 5]]
Optimal switches: 11
Job Order:  [5, 1, 2, 8, 9, 4, 7, 6

In [6]:
true_count = results_df['solution_found'].sum()
false_count = results_df.shape[0] - true_count

false_indices = results_df.index[results_df['solution_found'] == False].tolist()

print("True Count: ", true_count)
print("False Count: ", false_count)
print("False Indices: ", false_indices)

True Count:  330
False Count:  0
False Indices:  []


In [5]:
row_5 = df.iloc[5]
print(row_5['filename'])
num_jobs = row_5['num_jobs']
num_tools = row_5['num_tools']
magazine_capacity = row_5['magazine_capacity']
job_tools_requirements = row_5['job_tools']
filename = row_5['filename']
print("Number of Jobs: ", num_jobs)
print("Number of Tools: ", num_tools)
print("Magazine Capacity: ", magazine_capacity)
print("Job Tools Requirements: ", job_tools_requirements)
print("Filename: ", filename)

L1-5.txt
Number of Jobs:  9
Number of Tools:  15
Magazine Capacity:  5
Job Tools Requirements:  {1: [4, 8, 13], 2: [2, 3, 6, 12], 3: [1, 7, 10, 11, 14], 4: [6, 10], 5: [12, 15], 6: [1, 5, 11], 7: [5, 8, 10], 8: [8, 9], 9: [7, 9]}
Filename:  L1-5.txt


In [19]:
jobs = list(range(1, 10+1))
print(jobs)
num_nodes = list(range(0, len(jobs) + 3))
print(num_nodes)
job_nodes = list(range(1, len(jobs) + 1))
print(job_nodes)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [11]:
k = [1,2,3,4,5,6]
print(len(k))
print(list(range(0, len(k) + 3)))
print(list(range(1, len(k))))
START = 0
REPO = len(k) + 1
DETACHED = len(k) + 2
print(START)
print(REPO)
print(DETACHED)
print("----------------")
K = len(k) -1
print(K)
print(list(range(1, K)))

6
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 3, 4, 5]
0
7
8
----------------
5
[1, 2, 3, 4]
